# Classification of Liver Patients

Liver Patients are found using ILPD dataset

Importing Libraries

In [3]:
from copy import deepcopy
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
import numpy as np
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [52]:
def read_file(filename):
    df = pd.read_csv(filename, header = None)
    df[1], label1 = pd.factorize(df[1])
    return df

df = read_file('ILPD.csv')
print(df.head(5))


   0   1     2    3    4   5    6    7    8     9   10
0  65   0   0.7  0.1  187  16   18  6.8  3.3  0.90   1
1  62   1  10.9  5.5  699  64  100  7.5  3.2  0.74   1
2  62   1   7.3  4.1  490  60   68  7.0  3.3  0.89   1
3  58   1   1.0  0.4  182  14   20  6.8  3.4  1.00   1
4  72   1   3.9  2.0  195  27   59  7.3  2.4  0.40   1


In [71]:
def stratified_sampling(df, class1_p, class2_p):
    df_yes = df[df[10] == 1]
    df_no = df[df[10] == 2]
    df_yes[11] = np.random.uniform(0, 1, len(df_yes)) <= float(class1_p / 10)
    df_no[11] = np.random.uniform(0, 1, len(df_no)) <= float(class2_p/ 10)
    train = pd.DataFrame()
    test = pd.DataFrame()
    train1 = pd.DataFrame()
    train2 = pd.DataFrame()
    test1 = pd.DataFrame()
    test2 = pd.DataFrame()
    train1, test1 = df_yes[df_yes[11] == True], df_yes[df_yes[11] == False]
    train2, test2 = df_no[df_no[11] == True], df_no[df_no[11] == False]
    train = train1.append(train2)
    test = test1.append(test2)
    return train, test

train, test = stratified_sampling(df, 6, 9)
print(len(train), len(test))
    

413 166


C:\Users\Prashil\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Prashil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [72]:
def calc_metrics(pred, test):
    tp = tn = fp = fn = 0
    total = 0
    for x in test[10]:
        if x == pred[total]:
            if x == 1:
                tp = tp + 1
            else:
                tn = tn + 1

        else:
            if x == 1 and pred[total] == 2:
                fn = fn + 1
            if x == 2 and pred[total] == 1:
                fp = fp + 1
        total = total + 1

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    precision = tp / (tp + fp)
    
    return accuracy, sensitivity, specificity, precision

In [73]:
def get_pred(train, test, kernel):
    features = df.columns[0:10]
    y = train[10]
    C = 1.0
    clf = svm.SVC(kernel = kernel, gamma = 0.7, C=C)
    clf.fit(train[features], y)
    pred = clf.predict(test[features])
    
    return pred

In [74]:
pred_rbf = get_pred(train, test, 'rbf')
acc, sens, spec, prec = calc_metrics(pred_rbf, test)
print(acc, sens, spec, prec)

0.927710843373494 1.0 0.0 0.927710843373494


In [75]:
def get_all_metrics(kernel):
    all_acc = list()
    all_sens = list()
    all_spec = list()
    all_prec = list()
    for i in range(4, 10):
        acc_i = list()
        sens_i = list()
        spec_i = list()
        prec_i = list()
        for j in range(4, 10):
            acc = list()
            sens = list()
            spec = list()
            prec = list()
            for k in range(2):
                train, test = stratified_sampling(df, i, j)
                pred = get_pred(train, test, kernel)
                a, s, sp, p = calc_metrics(pred, test)
                #print(acc, sens, spec, prec)
                acc.append(a)
                sens.append(s)
                spec.append(sp)
                prec.append(p)
            acc_i.append(np.mean(acc))
            sens_i.append(np.mean(sens))
            spec_i.append(np.mean(spec))
            prec_i.append(np.mean(prec))
        all_acc.append(acc_i)
        all_sens.append(sens_i)
        all_spec.append(spec_i)
        all_prec.append(prec_i)
    return all_acc, all_sens, all_spec, all_prec

a, s, sp, pr = get_all_metrics('rbf')
print(a)         

C:\Users\Prashil\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Prashil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


[[0.71448627373482865, 0.75437731409391329, 0.79601702015495124, 0.82609589041095888, 0.88316041400395962, 0.93989672204759767], [0.693084693084693, 0.72282479848269321, 0.77256299275710805, 0.82184139784946242, 0.86172335600907024, 0.92116493301327429], [0.62689393939393945, 0.69589340082902318, 0.71074762425363724, 0.73812243033348568, 0.85864393338620149, 0.90048765997475066], [0.56806195216164013, 0.59764430428134552, 0.69819514946418493, 0.73122560745310339, 0.7803408399269629, 0.88131313131313127], [0.49406283499628928, 0.46522435897435899, 0.56887366818873675, 0.65861344537815125, 0.72342675832892644, 0.84892739273927398], [0.35115546218487392, 0.38915715539947326, 0.39632178049609246, 0.52551020408163263, 0.61462450592885376, 0.72692307692307689]]
